In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/movielens-20m-dataset/movie.csv
/kaggle/input/movielens-20m-dataset/tag.csv
/kaggle/input/movielens-20m-dataset/genome_tags.csv
/kaggle/input/movielens-20m-dataset/genome_scores.csv
/kaggle/input/movielens-20m-dataset/link.csv
/kaggle/input/movielens-20m-dataset/rating.csv


In [6]:
dir()

['In',
 'Out',
 '_',
 '_5',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_exit_code',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'dirname',
 'exit',
 'filename',
 'filenames',
 'get_ipython',
 'np',
 'os',
 'pd',
 'quit']

In [7]:
import numpy as np
import pandas as pd 
dataset=pd.read_csv("../input/movielens-20m-dataset/rating.csv")
mod_data = dataset.drop(columns="timestamp")

mod_data.userId -= 1
unique_movies = set(mod_data.movieId)
coded_movie = {}
count=0
for movie in unique_movies:
    coded_movie[movie] = count
    count+=1
mod_data["movieId"] = mod_data.apply(lambda row: coded_movie[row.movieId], axis=1)
#mod_data.to_csv("cleaned1",index =False)

#user_data =mod_data["userId"].value_counts().iloc[:2500].index.values
#movie_data = mod_data["movieId"].value_counts().iloc[:2500].index.values
#<----------------------------shrinking the dataset------------------------>

df  = mod_data

M = df.userId.max()+1

N = df.movie_idx.max()+1

from collections import Counter
n = 10000
m = 2500
user_id_max = Counter(df.userId)
movie_id_max = Counter(df.movie_idx)

user_list  = [u for u,c in user_id_max.most_common(n)]
movie_list = [u for u,c in movie_id_max.most_common(m)]

df_small =df[df.movie_idx.isin(movie_list) & df.userId.isin(user_list)].copy()
new_user_id_map = {}
i = 0
for old in user_list:
    
    new_user_id_map[old] = i
    i += 1
print("i:", i)

new_movie_id_map = {}
j = 0
for old in movie_list:
    new_movie_id_map[old] = j
    j += 1
print("j:", j)

#<-----------------------------creatinng dictionaries----------------------->
from random import shuffle
df = df_small.copy()
df = shuffle(df)
cutoff = int(0.8*len(df))
df_train = df.iloc[:cutoff]
df_test = df.iloc[cutoff:]

user2movie = {}
movie2user  = {}
usermovie2rating = {}

count = 0
def update(row):
  
    global count 
    count += 1
    if count % 100000 == 0:
        print("processed: %.3f" % (float(count)/cutoff))

    i = int(row.userId)
    j = int(row.movie_idx)
    if i not in user2movie:
        user2movie[i] = [j]
    else:
        user2movie[i].append(j)

    if j not in movie2user:
        movie2user[j] = [i]
    else:
        movie2user[j].append(i)

    usermovie2rating[(i,j)] = row.rating

df_train.apply(update,axis =1)    
    
usermovie2rating_test = {}
print("Calling: update_usermovie2rating_test")
count = 0
def update_usermovie2rating_test(row):
    global count
    count += 1
    if count % 100000 == 0:
        print("processed: %.3f" % (float(count)/len(df_test)))

    i = int(row.userId)
    j = int(row.movie_idx)
    usermovie2rating_test[(i,j)] = row.rating
df_test.apply(update_usermovie2rating_test, axis=1)
#<---------------------------------------creating the model------------------------------>
from sortedcontainers import SortedList
K = 25 # number of neighbors we'd like to consider
limit = 5 # number of common movies users must have in common in order to consider
N = max(list(user2movie.keys()))+1
m1 =  max(list(movie2user.keys()))
m2 = np.max([m for (u, m), r in usermovie2rating_test.items()])
M = max(m1,m2)+1
neighbors = [] # store neighbors in this list
averages = [] # each user's average rating for later use
deviations = [] # each user's deviation for later use

for i in range(N):
    movies_i = user2movie[i]
    movies_i_set  = set(movies_i)
    
    ratings_i = {movie : usermovie2rating[(i, movie)] for movie in movies_i}
    avg_i = np.mean(list(ratings_i.values()))
    dev_i = {movie : (usermovie2rating[(i,movie)]-avg_i) for movie in movies_i}
    dev_i_values = np.array(list(dev_i.values()))
    sigma_i = np.sqrt(dev_i_values.dot(dev_i_values))
    
    averages.append(avg_i)
    deviations.append(dev_i)
    sl = SortedList()
    for j in range(N):
        if(i!=j):
            movies_j = user2movie[j]
            movies_j_set = set(movies_j)
            common_movies = (movies_i_set & movies_j_set) # intersection
            if len(common_movies) > limit:
                ratings_j = {movie : usermovie2rating[(j, movie)] for movie in movies_j}
                avg_j = np.mean(list(ratings_j.values()))
                dev_j = {movie : (usermovie2rating[(j,movie)]-avg_j) for movie in movies_j}
                dev_j_values = np.array(list(dev_j.values()))
                sigma_j = np.sqrt(dev_j_values.dot(dev_j_values))
                
                numerator = sum(dev_i[m]*dev_j[m] for m in common_movies)
                w_ij = numerator / (sigma_i * sigma_j)
                sl.add((-w_ij, j))
                if len(sl) > K:
                    del sl[-1]
    neighbors.append(sl)
    if i % 1 == 0:
        print(i)

KeyboardInterrupt: 

In [8]:
import numpy as np
import pandas as pd 
dataset=pd.read_csv("../input/movielens-20m-dataset/rating.csv")
mod_data = dataset.drop(columns="timestamp")

In [11]:
mod_data.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [12]:
mod_data.userId -= 1
unique_movies = set(mod_data.movieId)
coded_movie = {}
count=0
for movie in unique_movies:
    coded_movie[movie] = count
    count+=1
mod_data["movie_idx"] = mod_data.apply(lambda row: coded_movie[row.movieId], axis=1)

In [13]:
mod_data.head()

,userId,movieId,rating,movie_idx
0,0,2,3.5,2
1,0,29,3.5,29
2,0,32,3.5,32
3,0,47,3.5,47
4,0,50,3.5,50


In [14]:
df  = mod_data

M = df.userId.max()+1

N = df.movie_idx.max()+1

from collections import Counter
n = 10000
m = 2500
user_id_max = Counter(df.userId)
movie_id_max = Counter(df.movie_idx)

user_list  = [u for u,c in user_id_max.most_common(n)]
movie_list = [u for u,c in movie_id_max.most_common(m)]

df_small =df[df.movie_idx.isin(movie_list) & df.userId.isin(user_list)].copy()


In [15]:
len(df_small)

5842741

In [16]:
new_user_id_map = {}
i = 0
for old in user_list:
    
    new_user_id_map[old] = i
    i += 1
print("i:", i)

new_movie_id_map = {}
j = 0
for old in movie_list:
    new_movie_id_map[old] = j
    j += 1
print("j:", j)


i: 10000
j: 2500


In [32]:
print("Setting new ids")
df_small.loc[:, 'userId'] = df_small.apply(lambda row: new_user_id_map[row.userId], axis=1)
df_small.loc[:, 'movie_idx'] = df_small.apply(lambda row: new_movie_id_map[row.movie_idx], axis=1)
# df_small.drop(columns=['userId', 'movie_idx'])
# df_small.rename(index=str, columns={'new_userId': 'userId', 'new_movie_idx': 'movie_idx'})
print("max user id:", df_small.userId.max())
print("max movie id:", df_small.movie_idx.max())

print("small dataframe size:", len(df_small))

Setting new ids
max user id: 9999
max movie id: 2499
small dataframe size: 5842741


In [33]:
from sklearn.utils import shuffle
df = df_small.copy()
df = shuffle(df)
cutoff = int(0.8*len(df))
df_train = df.iloc[:cutoff]
df_test = df.iloc[cutoff:]

user2movie = {}
movie2user  = {}
usermovie2rating = {}

count = 0

In [34]:
def update(row):
  
    global count 
    count += 1
    if count % 100000 == 0:
        print("processed: %.3f" % (float(count)/cutoff))

    i = int(row.userId)
    j = int(row.movie_idx)
    if i not in user2movie:
        user2movie[i] = [j]
    else:
        user2movie[i].append(j)

    if j not in movie2user:
        movie2user[j] = [i]
    else:
        movie2user[j].append(i)

    usermovie2rating[(i,j)] = row.rating


In [35]:
df_train.apply(update,axis =1)    


processed: 0.021
processed: 0.043
processed: 0.064
processed: 0.086
processed: 0.107
processed: 0.128
processed: 0.150
processed: 0.171
processed: 0.193
processed: 0.214
processed: 0.235
processed: 0.257
processed: 0.278
processed: 0.300
processed: 0.321
processed: 0.342
processed: 0.364
processed: 0.385
processed: 0.406
processed: 0.428
processed: 0.449
processed: 0.471
processed: 0.492
processed: 0.513
processed: 0.535
processed: 0.556
processed: 0.578
processed: 0.599
processed: 0.620
processed: 0.642
processed: 0.663
processed: 0.685
processed: 0.706
processed: 0.727
processed: 0.749
processed: 0.770
processed: 0.792
processed: 0.813
processed: 0.834
processed: 0.856
processed: 0.877
processed: 0.899
processed: 0.920
processed: 0.941
processed: 0.963
processed: 0.984


3572497     None
10769401    None
9210821     None
8047506     None
18703298    None
            ... 
5374920     None
2914630     None
6482525     None
14845347    None
6142447     None
Length: 4674192, dtype: object

In [40]:
max(list(df_train.userId))
#df_train.head()

9999

In [41]:
usermovie2rating_test = {}
print("Calling: update_usermovie2rating_test")
count = 0
def update_usermovie2rating_test(row):
    global count
    count += 1
    if count % 100000 == 0:
        print("processed: %.3f" % (float(count)/len(df_test)))

    i = int(row.userId)
    j = int(row.movie_idx)
    usermovie2rating_test[(i,j)] = row.rating
df_test.apply(update_usermovie2rating_test, axis=1)

Calling: update_usermovie2rating_test
processed: 0.086
processed: 0.171
processed: 0.257
processed: 0.342
processed: 0.428
processed: 0.513
processed: 0.599
processed: 0.685
processed: 0.770
processed: 0.856
processed: 0.941


903858      None
8662783     None
7645478     None
19524150    None
6785999     None
            ... 
12423318    None
4031209     None
8821782     None
8054436     None
5716761     None
Length: 1168549, dtype: object

In [43]:
from sortedcontainers import SortedList
K = 25 # number of neighbors we'd like to consider
limit = 5 # number of common movies users must have in common in order to consider
N = max(list(user2movie.keys()))+1
m1 =  max(list(movie2user.keys()))
m2 = np.max([m for (u, m), r in usermovie2rating_test.items()])
M = max(m1,m2)+1
neighbors = [] # store neighbors in this list
averages = [] # each user's average rating for later use
deviations = [] # each user's deviation for later use


In [44]:
N,M

(10000, 2500)

In [ ]:
for i in range(N):
    movies_i = user2movie[i]
    movies_i_set  = set(movies_i)
    
    ratings_i = {movie : usermovie2rating[(i, movie)] for movie in movies_i}
    avg_i = np.mean(list(ratings_i.values()))
    dev_i = {movie : (usermovie2rating[(i,movie)]-avg_i) for movie in movies_i}
    dev_i_values = np.array(list(dev_i.values()))
    sigma_i = np.sqrt(dev_i_values.dot(dev_i_values))
    
    averages.append(avg_i)
    deviations.append(dev_i)
    sl = SortedList()
    for j in range(N):
        if(i!=j):
            movies_j = user2movie[j]
            movies_j_set = set(movies_j)
            common_movies = (movies_i_set & movies_j_set) # intersection
            if len(common_movies) > limit:
                ratings_j = {movie : usermovie2rating[(j, movie)] for movie in movies_j}
                avg_j = np.mean(list(ratings_j.values()))
                dev_j = {movie : (usermovie2rating[(j,movie)]-avg_j) for movie in movies_j}
                dev_j_values = np.array(list(dev_j.values()))
                sigma_j = np.sqrt(dev_j_values.dot(dev_j_values))
                
                numerator = sum(dev_i[m]*dev_j[m] for m in common_movies)
                w_ij = numerator / (sigma_i * sigma_j)
                sl.add((-w_ij, j))
                if len(sl) > K:
                    del sl[-1]
    neighbors.append(sl)
    if i % 1 == 0:
        print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
